In [ ]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_socketio import SocketIO, send
from flask_cors import CORS
import datetime
import stripe
import logging

# Initialize app
app = Flask(UTA_EVENTS)
CORS(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'postgresql://user:password@db_host/db_name'
app.config['JWT_SECRET_KEY'] = 'your_jwt_secret'
app.config['STRIPE_SECRET_KEY'] = 'your_stripe_secret_key'
db = SQLAlchemy(app)
jwt = JWTManager(app)
socketio = SocketIO(app, cors_allowed_origins='*')
stripe.api_key = app.config['STRIPE_SECRET_KEY']

# Setup Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Models
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

class Event(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    date = db.Column(db.DateTime, nullable=False)
    location = db.Column(db.String(200), nullable=False)
    description = db.Column(db.Text, nullable=True)

class Agenda(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    event_id = db.Column(db.Integer, db.ForeignKey('event.id'), nullable=False)
    title = db.Column(db.String(200), nullable=False)
    time = db.Column(db.DateTime, nullable=False)
    speaker = db.Column(db.String(100), nullable=True)

class Payment(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    event_id = db.Column(db.Integer, db.ForeignKey('event.id'), nullable=False)
    amount = db.Column(db.Float, nullable=False)
    status = db.Column(db.String(50), nullable=False)

# Routes
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    new_user = User(username=data['username'], password=data['password'])
    db.session.add(new_user)
    db.session.commit()
    logger.info(f'User {data["username"]} registered successfully')
    return jsonify({'message': 'User registered successfully'})

@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    user = User.query.filter_by(username=data['username'], password=data['password']).first()
    if not user:
        return jsonify({'message': 'Invalid credentials'}), 401
    access_token = create_access_token(identity=user.id, expires_delta=datetime.timedelta(hours=1))
    return jsonify({'access_token': access_token})

@app.route('/events', methods=['POST'])
@jwt_required()
def create_event():
    data = request.get_json()
    new_event = Event(name=data['name'], date=datetime.datetime.strptime(data['date'], '%Y-%m-%d'), location=data['location'], description=data.get('description'))
    db.session.add(new_event)
    db.session.commit()
    logger.info(f'Event {data["name"]} created')
    return jsonify({'message': 'Event created'})

@app.route('/events', methods=['GET'])
def get_events():
    events = Event.query.all()
    return jsonify([{'id': event.id, 'name': event.name, 'date': event.date.strftime('%Y-%m-%d'), 'location': event.location, 'description': event.description} for event in events])

@app.route('/agenda/<int:event_id>', methods=['GET'])
def get_agenda(event_id):
    agenda = Agenda.query.filter_by(event_id=event_id).all()
    return jsonify([{'id': item.id, 'title': item.title, 'time': item.time.strftime('%Y-%m-%d %H:%M'), 'speaker': item.speaker} for item in agenda])

@app.route('/pay', methods=['POST'])
@jwt_required()
def make_payment():
    data = request.get_json()
    user_id = get_jwt_identity()
    event = Event.query.get(data['event_id'])
    if not event:
        return jsonify({'message': 'Event not found'}), 404
    payment_intent = stripe.PaymentIntent.create(amount=int(data['amount'] * 100), currency='usd')
    new_payment = Payment(user_id=user_id, event_id=event.id, amount=data['amount'], status='pending')
    db.session.add(new_payment)
    db.session.commit()
    logger.info(f'Payment of ${data["amount"]} initiated for user {user_id} for event {event.id}')
    return jsonify({'client_secret': payment_intent.client_secret})

@socketio.on('message')
def handle_message(msg):
    send(msg, broadcast=True)
    logger.info(f'New chat message: {msg}')

if __name__ == '__main__':
    db.create_all()
    socketio.run(app, debug=False, host='0.0.0.0', port=5000)
